In [1]:
from __future__ import print_function, division
import pylab as pl
import pandas as pd
import numpy as np
import scipy.stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from numpy import inf

%pylab inline
try:
    import urllib2 as urllib
except ImportError:
    import urllib.request as urllib

import io
import os
import zipfile

#function to download from the url

def getCSV(url, cache_name=None, is_zip=False,**kwargs):
    cache_name = cache_name or os.path.basename(url)
    if is_zip:
            # Get zip buffer
            remotezip = urllib.urlopen(url).read()
            # Open in memory
            with zipfile.ZipFile(io.BytesIO(remotezip)) as z:
                # Read the filename or the first file if there are naming inconsistencies
                read_zip_file = cache_name if cache_name in z.namelist() else z.namelist()[0]
                df = pd.read_csv(z.open(read_zip_file),**kwargs)
    else:
        df = pd.read_csv(urllib.urlopen(url),**kwargs)
    return df

Populating the interactive namespace from numpy and matplotlib


/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
#downloaded data and uploaded onto compute
df= pd.read_csv("mta_data.csv")

In [3]:
df.shape

(10267810, 12)

In [4]:
df.head()

,Unnamed: 0,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/24/2016,03:00:00,REGULAR,5980509,2025806
1,1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/24/2016,07:00:00,REGULAR,5980531,2025818
2,2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/24/2016,11:00:00,REGULAR,5980613,2025912
3,3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/24/2016,15:00:00,REGULAR,5980899,2025965
4,4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/24/2016,19:00:00,REGULAR,5981382,2026001


In [5]:
# first check for columns with null values
df.isnull().any()

Unnamed: 0                                                              False
C/A                                                                     False
UNIT                                                                    False
SCP                                                                     False
STATION                                                                 False
LINENAME                                                                False
DIVISION                                                                False
DATE                                                                    False
TIME                                                                    False
DESC                                                                    False
ENTRIES                                                                 False
EXITS                                                                   False
dtype: bool

In [6]:
df = df.dropna()
df.isnull().any()

Unnamed: 0                                                              False
C/A                                                                     False
UNIT                                                                    False
SCP                                                                     False
STATION                                                                 False
LINENAME                                                                False
DIVISION                                                                False
DATE                                                                    False
TIME                                                                    False
DESC                                                                    False
ENTRIES                                                                 False
EXITS                                                                   False
dtype: bool

In [7]:
#check dtype
df.dtypes

Unnamed: 0                                                               int64
C/A                                                                     object
UNIT                                                                    object
SCP                                                                     object
STATION                                                                 object
LINENAME                                                                object
DIVISION                                                                object
DATE                                                                    object
TIME                                                                    object
DESC                                                                    object
ENTRIES                                                                  int64
EXITS                                                                    int64
dtype: object

In [8]:
# check for unique time values 
df.TIME.unique()

array(['03:00:00', '07:00:00', '11:00:00', ..., '13:12:52', '17:24:52',
       '07:54:01'], dtype=object)

As the records are very different, I will strip time to include only hour before sorting them into time interval bins

In [9]:
df['TIME'] = df['TIME'].str.split(':').str[0]
df['TIME'] = df['TIME'].astype(int)

Sort times into intervals:
1. 0-3
2. 4-7
3. 8-11
4. 12-15
5. 16-19
6. 20-23



In [10]:
# sorting into time interval bins
df['TIMEINT'] = pd.cut(df['TIME'], bins=[0,4,8,12,16,20,300], labels=[1,2,3,4,5,6])
df.head()

,Unnamed: 0,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,TIMEINT
0,0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/24/2016,3,REGULAR,5980509,2025806,1
1,1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/24/2016,7,REGULAR,5980531,2025818,2
2,2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/24/2016,11,REGULAR,5980613,2025912,3
3,3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/24/2016,15,REGULAR,5980899,2025965,4
4,4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/24/2016,19,REGULAR,5981382,2026001,5


In [11]:
# now groupby station and time to aggregate commuter entries per station 
df1= df.groupby(['STATION','DATE','TIMEINT'])['ENTRIES'].sum()
df1

STATION    DATE        TIMEINT
1 AV       01/01/2016  1          1407774726
                       2          1407775796
                       3          1407776768
                       4          1407779799
                       5          1407783611
                       6          1407786852
           01/02/2016  1          1407788395
                       2          1407788714
                       3          1407790321
                       4          1407794225
                       5          1407798688
                       6          1407802820
           01/03/2016  1          1407805211
                       2          1407805739
                       3          1407807196
                       4          1407810515
                       5          1407814293
                       6          1407817257
           01/04/2016  1          1407818153
                       2          1407818810
                       3          1407824411
                       4

In [12]:
# convert series to dataframe 
df1 = df1.to_frame().reset_index()
df1

,STATION,DATE,TIMEINT,ENTRIES
0,1 AV,01/01/2016,1,1407774726
1,1 AV,01/01/2016,2,1407775796
2,1 AV,01/01/2016,3,1407776768
3,1 AV,01/01/2016,4,1407779799
4,1 AV,01/01/2016,5,1407783611
5,1 AV,01/01/2016,6,1407786852
6,1 AV,01/02/2016,1,1407788395
7,1 AV,01/02/2016,2,1407788714
8,1 AV,01/02/2016,3,1407790321
9,1 AV,01/02/2016,4,1407794225


In [13]:
df1.to_csv('mta_combined.csv',encoding='utf-8')
